## Developing an Embedding Layer

Essentially an embedding layer is a mapping from discrete objects, such as words, to points in a vector space. The whole purpose of an embedding layer it is to transform our input data into a format that the decoder of our GPT model can parse. Notice that the choice of discrete objects is ours, we can choose words, sentences, paragraphs, etc. We will focus on word embeddings.

## Word Embeddings

Because of the popularity of word embeddings, several algorithms have been develop that transform a word into a vector. One famous algorithm is **Word2Vec**. This algorithm  will turn words into vectors in such a way that words that have similar semantic meaning will be mapped to vectors that are close to each other. 
Word2Vec utilizes a simple neural network to achieve this. 

We can use a pretrained model like Word2Vec for our Embedding layer, but LLMs commonly produce their own embeddings that are part of the input layer and are updated while training. The advantage of this approach is that the embeddings are optimized for the specific task and given data.